Q4 (linechart)
county, state, shootings, year

In [2]:
import pandas as pd
import altair as alt

In [3]:
df_shootings = pd.read_csv("datasets/MassShootingsComplete_FIPS.csv")
df_population = pd.read_csv("datasets/CountyPopulationAllYears.csv")

In [ ]:
'''
CHART 1: total shootings / year   +   governement (democratic / republican)
'''
def chart_Q4():
    df_shootings = pd.read_csv("datasets/MassShootingsComplete_FIPS.csv")
    gov = {
        'Year': [i for i in range(2014, 2025)],
        'y1': [230]*11,
        'y2': [220]*11,
        'governement': ['Democratic']*3 + ['Republican']*5 + ['Democratic']*3
    }

    # Convert to a DataFrame
    df = pd.DataFrame(gov)

    # Create the Altair bar chart using 'y1' or 'y2' as the Total Shootings value
    chart_gov = alt.Chart(df).mark_area().encode(
        x=alt.X('Year:O', title='Year', axis=alt.Axis(labelAngle=0)),
        y=alt.Y('y1:Q', title='Total Shootings', scale=alt.Scale(domain=(200, 710))),
        y2='y2:Q',  # Specify the y2 encoding for the upper boundary of the area
        color=alt.Color('governement', scale=alt.Scale(domain=['Democratic', 'Republican'], range=['#3182bd', '#d7301f']))
    ).properties(
        title='Total Shootings per Year in the USA',
        width=500,
        height=400
    )


    # Converteix la data a un format de data de pandas
    df_shootings['Incident Date'] = pd.to_datetime(df_shootings['Incident Date'])

    # Extreu l'any de la data de l'incident
    df_shootings['Year'] = df_shootings['Incident Date'].dt.year

    # Agrupa per any i compta el nombre de tirotejos
    shootings_per_year = df_shootings.groupby('Year').size().reset_index(name='Total Shootings')

    # Crea el gràfic de línia amb Altair
    chart_ms = alt.Chart(shootings_per_year).mark_line(
        color='black',
        point=alt.OverlayMarkDef(filled=True, fill="black"),
        strokeWidth=2.5).encode(
        x=alt.X('Year:O', title='Year', axis=alt.Axis(labelAngle=0)),  # Usa ':O' per especificar que l'eix X és ordinal
        y=alt.Y('Total Shootings:Q', title='Total Shootings').scale(domain=(200,710))
    ).properties(
        title='Total Shootings per Year in the USA',
        width=500,
        height=400
    )


    return chart_gov + chart_ms

chart = chart_Q4()
chart.show()

alt.LayerChart(...)

In [50]:
'''
CHART 2: total shootings / year and / county
'''


shootings_per_state_year = df_shootings.groupby(['State', 'Year']).size().reset_index(name='Total Shootings')

chart = alt.Chart(shootings_per_state_year).mark_line().encode(
    x=alt.X('Year:O', title='Year'),  # Usa ':O' per especificar que l'eix X és ordinal
    y=alt.Y('Total Shootings:Q', title='Total Shootings'),
    color='State:N'  # Utilitza 'State' per distingir cada línia amb un color diferent
).properties(
    title='Total Shootings per Year per State in the USA',
    width=500,
    height=400
)

chart.show()

#not feasable... xdd

alt.Chart(...)

In [46]:
years = range(2014, 2024)

# Comptar el total de tirotejos per cada any
df_shootings['Incident Date'] = pd.to_datetime(df_shootings['Incident Date'])
df_shootings['Year'] = df_shootings['Incident Date'].dt.year
shootings_totals = df_shootings.groupby("Year").size().reindex(years, fill_value=0)

# Calcular la població total per cada any al dataset de població
population_totals = df_population[[f"POPESTIMATE{year}" for year in years]].sum()

# Calcular la ràtio tirotejos/població per cada any
ratio = shootings_totals.values * 1e6 / population_totals.values

# Crear un DataFrame amb els resultats
data = pd.DataFrame({
    'Year': years,
    'Shootings per capita': ratio
})

chart_ms = alt.Chart(data).mark_line(
    color='black',
    point=alt.OverlayMarkDef(filled=True, fill="black"),
    strokeWidth=2.5).encode(
    x=alt.X('Year:O', title='Year', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('Shootings per capita:Q', title='Total Shootings', scale=alt.Scale(domain=(0.3,1.05)))
).properties(
    title='Shootings per Capita *10e6  in the USA',
    width=500,
    height=400
)

gov = {
    'Year': [i for i in range(2014, 2025)],
    'y1': [0.3+0.042]*11,
    'y2': [0.4-0.042]*11,
    'governement': ['Democratic']*3 + ['Republican']*5 + ['Democratic']*3
}

# Convert to a DataFrame
df = pd.DataFrame(gov)

# Create the Altair bar chart using 'y1' or 'y2' as the Total Shootings value
chart_gov = alt.Chart(df).mark_area().encode(
    x=alt.X('Year:O', title='Year', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('y1:Q', title='Total Shootings'),
    y2='y2:Q',  # Specify the y2 encoding for the upper boundary of the area
    color=alt.Color('governement', scale=alt.Scale(domain=['Democratic', 'Republican'], range=['#3182bd', '#d7301f']))
).properties(
    title='Total Shootings per Year in the USA',
    width=500,
    height=400
)


chart_ms + chart_gov

alt.LayerChart(...)